# GOLD LAYER - AGREGAÇÕES

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable
from datetime import datetime

# CONFIGURAÇÃO

In [0]:
STORAGE_ACCOUNT = "mystoacc"
SILVER_PATH = f"abfss://silver@{STORAGE_ACCOUNT}.dfs.core.windows.net"
GOLD_PATH = f"abfss://gold@{STORAGE_ACCOUNT}.dfs.core.windows.net"

spark.sql("USE CATALOG hive_metastore")
spark.sql("CREATE DATABASE IF NOT EXISTS healthcare_gold LOCATION 'abfss://gold@mystoacc.dfs.core.windows.net/'")
spark.sql("USE healthcare_gold")

PROCESSING_TIMESTAMP = datetime.now()
PROCESSING_DATE = PROCESSING_TIMESTAMP.strftime("%Y-%m-%d")

print(f"Processamento Gold iniciado: {PROCESSING_TIMESTAMP}")
print("Criando agregações de negócio...")

In [0]:
# FUNÇÕES AUXILIARES

In [0]:
def add_gold_metadata(df: DataFrame) -> DataFrame:
    """Adiciona colunas de metadados da Gold"""
    return df.withColumn("gold_processed_at", lit(PROCESSING_TIMESTAMP)) \
             .withColumn("gold_processing_date", lit(PROCESSING_DATE))

def log_aggregation(table_name: str, record_count: int):
    """Loga métricas de agregação"""
    print(f"\n{table_name}:")
    print(f"  Registros agregados: {record_count:,}")

def save_to_gold(df: DataFrame, table_name: str) -> dict:
    """
    Salva DataFrame na Gold com modo OVERWRITE
    (Agregações são recalculadas do zero a cada execução)
    """
    try:
        gold_path = f"{GOLD_PATH}/{table_name}"
        
        df_final = add_gold_metadata(df)
        
        record_count = df_final.count()
        log_aggregation(table_name, record_count)
        
        df_final.write.format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable(f"healthcare_gold.{table_name}")
        
        print(f"✓ Tabela {table_name} salva na Gold")
        
        return {
            "table": table_name,
            "status": "SUCCESS",
            "records": record_count
        }
        
    except Exception as e:
        print(f"✗ ERRO ao salvar {table_name}: {str(e)}")
        return {
            "table": table_name,
            "status": "FAILED",
            "error": str(e)
        }

# CARREGAR DADOS DA SILVER

In [0]:
df_consultas = spark.read.format("delta").load(f"{SILVER_PATH}/fato_consultas")

df_data = spark.read.format("delta").load(f"{SILVER_PATH}/dim_data")
df_medico = spark.read.format("delta").load(f"{SILVER_PATH}/dim_medico")
df_clinica = spark.read.format("delta").load(f"{SILVER_PATH}/dim_clinica")
df_diagnostico = spark.read.format("delta").load(f"{SILVER_PATH}/dim_diagnostico")
df_exame = spark.read.format("delta").load(f"{SILVER_PATH}/dim_exame")
df_paciente = spark.read.format("delta").load(f"{SILVER_PATH}/dim_paciente")

print("✓ Dados carregados")

print("\n" + "="*60)
print("Criando: agg_consultas_por_periodo")
print("="*60)

agg_periodo = df_consultas.alias("f") \
    .join(df_data.alias("d"), col("f.sk_data") == col("d.sk_data")) \
    .groupBy(
        col("d.ano"),
        col("d.mes"),
        col("d.data_completa")
    ) \
    .agg(
        count("f.id_consulta").alias("total_consultas"),
        countDistinct("f.sk_paciente").alias("pacientes_unicos"),
        countDistinct("f.sk_medico").alias("medicos_ativos"),
        sum("f.valor_total").alias("receita_total"),
        avg("f.valor_total").alias("ticket_medio"),
        sum(when(col("f.plano_cobriu") == 1, 1).otherwise(0)).alias("consultas_plano"),
        sum(when(col("f.plano_cobriu") == 0, 1).otherwise(0)).alias("consultas_particular")
    ) \
    .withColumn("percentual_plano", 
                round(col("consultas_plano") / col("total_consultas") * 100, 2)) \
    .orderBy("ano", "mes")

result_periodo = save_to_gold(agg_periodo, "agg_consultas_por_periodo")

print("\n" + "="*60)
print("Criando: agg_consultas_por_medico")
print("="*60)

agg_medico = df_consultas.alias("f") \
    .join(df_medico.alias("m"), col("f.sk_medico") == col("m.sk_medico")) \
    .groupBy(
        col("m.sk_medico"),
        col("m.especialidade"),
        col("m.uf_crm")
    ) \
    .agg(
        count("f.id_consulta").alias("total_consultas"),
        countDistinct("f.sk_paciente").alias("pacientes_atendidos"),
        sum("f.valor_total").alias("receita_gerada"),
        avg("f.valor_total").alias("ticket_medio"),
        min("f.valor_total").alias("consulta_min"),
        max("f.valor_total").alias("consulta_max")
    ) \
    .withColumn("receita_por_paciente", 
                round(col("receita_gerada") / col("pacientes_atendidos"), 2)) \
    .orderBy(col("total_consultas").desc())

result_medico = save_to_gold(agg_medico, "agg_consultas_por_medico")

print("\n" + "="*60)
print("Criando: agg_consultas_por_clinica")
print("="*60)

agg_clinica = df_consultas.alias("f") \
    .join(df_clinica.alias("c"), col("f.sk_clinica") == col("c.sk_clinica")) \
    .groupBy(
        col("c.sk_clinica"),
        col("c.tipo_clinica"),
        col("c.estado")
    ) \
    .agg(
        count("f.id_consulta").alias("total_consultas"),
        countDistinct("f.sk_paciente").alias("pacientes_atendidos"),
        countDistinct("f.sk_medico").alias("medicos_atuantes"),
        sum("f.valor_total").alias("receita_total"),
        avg("f.valor_total").alias("ticket_medio")
    ) \
    .withColumn("consultas_por_medico", 
                round(col("total_consultas") / col("medicos_atuantes"), 2)) \
    .orderBy(col("receita_total").desc())

result_clinica = save_to_gold(agg_clinica, "agg_consultas_por_clinica")

# AGREGAÇÃO 4: CONSULTAS POR DIAGNÓSTICO

In [0]:
print("\n" + "="*60)
print("Criando: agg_consultas_por_diagnostico")
print("="*60)

agg_diagnostico = df_consultas.alias("f") \
    .join(df_diagnostico.alias("diag"), col("f.sk_diagnostico") == col("diag.sk_diagnostico")) \
    .groupBy(
        col("diag.sk_diagnostico"),
        col("diag.codigo_cid"),
        col("diag.descricao_cid")
    ) \
    .agg(
        count("f.id_consulta").alias("total_casos"),
        countDistinct("f.sk_paciente").alias("pacientes_afetados"),
        sum("f.valor_total").alias("custo_total"),
        avg("f.valor_total").alias("custo_medio_tratamento")
    ) \
    .orderBy(col("total_casos").desc())

result_diagnostico = save_to_gold(agg_diagnostico, "agg_consultas_por_diagnostico")


# AGREGAÇÃO 5: PERFORMANCE DE EXAMES

In [0]:
print("\n" + "="*60)
print("Criando: agg_performance_exames")
print("="*60)

agg_exame = df_consultas.alias("f") \
    .join(df_exame.alias("e"), col("f.sk_exame") == col("e.sk_exame")) \
    .groupBy(
        col("e.sk_exame"),
        col("e.tipo_exame"),
        col("e.categoria_exame")
    ) \
    .agg(
        count("f.id_consulta").alias("total_realizados"),
        countDistinct("f.sk_paciente").alias("pacientes_distintos"),
        sum("f.valor_total").alias("receita_exames"),
        avg("f.valor_total").alias("valor_medio_exame")
    ) \
    .orderBy(col("total_realizados").desc())

result_exame = save_to_gold(agg_exame, "agg_performance_exames")

# AGREGAÇÃO 6: RESUMO POR ESPECIALIDADE

In [0]:
print("\n" + "="*60)
print("Criando: agg_resumo_especialidade")
print("="*60)

agg_especialidade = df_consultas.alias("f") \
    .join(df_medico.alias("m"), col("f.sk_medico") == col("m.sk_medico")) \
    .groupBy(col("m.especialidade")) \
    .agg(
        count("f.id_consulta").alias("total_consultas"),
        countDistinct("f.sk_medico").alias("medicos_ativos"),
        countDistinct("f.sk_paciente").alias("pacientes_atendidos"),
        sum("f.valor_total").alias("receita_total"),
        avg("f.valor_total").alias("ticket_medio")
    ) \
    .withColumn("consultas_por_medico", 
                round(col("total_consultas") / col("medicos_ativos"), 2)) \
    .orderBy(col("receita_total").desc())


result_especialidade = save_to_gold(agg_especialidade, "agg_resumo_especialidade")


# AGREGAÇÃO 7: ANÁLISE DE PACIENTES

In [0]:
print("\n" + "="*60)
print("Criando: agg_perfil_pacientes")
print("="*60)

current_year = datetime.now().year

agg_pacientes = df_consultas.alias("f") \
    .join(df_paciente.alias("p"), col("f.sk_paciente") == col("p.sk_paciente")) \
    .withColumn("idade_aproximada", lit(current_year) - col("p.ano_nascimento")) \
    .withColumn("faixa_etaria", 
                when(col("idade_aproximada") < 18, "0-17")
                .when(col("idade_aproximada") < 30, "18-29")
                .when(col("idade_aproximada") < 45, "30-44")
                .when(col("idade_aproximada") < 60, "45-59")
                .otherwise("60+")) \
    .groupBy(
        col("p.sexo"),
        col("faixa_etaria"),
        col("p.cidade")
    ) \
    .agg(
        countDistinct("f.sk_paciente").alias("total_pacientes"),
        count("f.id_consulta").alias("total_consultas"),
        sum("f.valor_total").alias("gasto_total"),
        avg("f.valor_total").alias("gasto_medio_consulta")
    ) \
    .withColumn("consultas_por_paciente", 
                round(col("total_consultas") / col("total_pacientes"), 2)) \
    .orderBy("sexo", "faixa_etaria")

result_pacientes = save_to_gold(agg_pacientes, "agg_perfil_pacientes")

# SUMÁRIO DE EXECUÇÃO

In [0]:
import pandas as pd
import builtins

results = [
    result_periodo,
    result_medico,
    result_clinica,
    result_diagnostico,
    result_exame,
    result_especialidade,
    result_pacientes
]

results_df = pd.DataFrame(results)
print("\n" + "="*80)
print("SUMÁRIO DE EXECUÇÃO - GOLD AGREGAÇÕES")
print("="*80)
display(results_df)

total_tables = len(results)
success_count = len([r for r in results if r['status'] == 'SUCCESS'])
failed_count = len([r for r in results if r['status'] == 'FAILED'])

if success_count > 0:
    total_records = builtins.sum([r.get('records', 0) for r in results if r['status'] == 'SUCCESS'])
    print(f"\nTabelas agregadas criadas: {success_count}/{total_tables}")
    print(f"Total de registros agregados: {total_records:,}")

if failed_count > 0:
    print(f"\n⚠ ALERTA: {failed_count} agregações falharam!")

print("\n" + "="*80)
print("✓ Camada Gold - Agregações concluída!")
print("="*80)